In [35]:
import random
import numpy as np
import io, os, sys, types
from ipynb.fs.full.Bloque_CanalAWGN import EjecucionCanal
from ipynb.fs.full.Bloque_CanalAWGN import ConvDec
import pandas as pd

DETECCION DE ERRORES

In [36]:
def detect_errores(dato_received, dato_send):
    dato_corregido = []
    error_vector = []
    errores = 0
    for x in range (len(dato_received)):
        xor = int(dato_send[x]) ^ int(dato_received[x])             #deteccion de errores por medio del xor
        error_vector.append(xor)
        if (xor == 1):                                                 #correcion de errores
            errores = errores + 1
            if int(dato_received[x]) == 0:
                dato_corregido.append('1')

            elif int(dato_received[x]) == 1:
                dato_corregido.append('0')
        else: 
            dato_corregido.append(dato_received[x])
    return [error_vector, dato_corregido, errores]
    

CALCULO DEL BER

In [37]:
def BER(error_vector,errores):
    tamaño = (len(error_vector))
    ber = errores / tamaño
    return ber

DECODIFICACION DE LOS DATOS

In [38]:
def decodificador(dato_corregido):
    BPM = 0
    if(len(dato_corregido)==7):
        bits_msj = dato_corregido[0:4]
        bits_parity = dato_corregido[5:7]
        bits_msj_str = ''.join(str(x) for x in bits_msj)
        x = int(bits_msj_str,2)
        if (x==0):
            BPM = random.randint(0, 80) # valores < 80 mapean a 0
        elif (x==1):
            BPM = random.randint(80, 90) #   80 < valores < 90 mapean a 1
        elif (x==2):
            BPM = random.randint(90, 100) #  90 < valores < 100 mapean a 2
        elif (x==3):
            BPM = random.randint(100, 110) #  100 < valores < 110 mapean a 3 
        elif (x==4):
            BPM = random.randint(110, 120) #  110 < valores < 120 mapean a 4
        elif (x==5):
            BPM = random.randint(120, 130) #  120 < valores < 130 mepean a 5
        elif (x==6):
            BPM = random.randint(130, 140) #  130 < valores < 140 mapean a 6
        elif (x==7):
            BPM = random.randint(140, 150) #  140 < valores < 150 mapean a 7
        elif (x==8):
            BPM = random.randint(150, 160) #  150 < valores < 160 mapean a 8
        elif (x==9):
            BPM = random.randint(160, 170) #  160 < valores < 170 mapean a 9
        elif (x==10):
            BPM = random.randint(170, 200) # valores > 170 mapean a 10

    elif(len(dato_corregido)==15):
        bits_msj = dato_corregido[0:11]
        bits_parity = dato_corregido[12:15]
        bits_msj_str = ''.join(str(x) for x in bits_msj)
        BPM = int(bits_msj_str,2)
        
    return BPM

In [39]:
with open('datacod15.txt') as f:
    file_señales_enviadas = f.readlines()
    
with open('Canalconruido.txt') as f:
    file_señales_recibidas = f.readlines()

error_vector_file = open("VectorErrores.txt", "w") #Creación del archivo txt
BER_file = open("BER.txt", "w") #Creación del archivo txt
BPM_file = open("BPM.txt", "w") #Creación del archivo txt
    
for x in range (len(file_señales_recibidas)):
    if (x == 0):
        continue
    else:
        dato_send = file_señales_enviadas[x].split()
        dato_received = file_señales_recibidas[x].split()    

        #dato_send = ['0','0','0','0','1','1','1','1','0','1','0','0','1','1','1'] #prueba ['1','0','0','0','0','1','0']
        #dato_received = ['0','0','0','1','1','1','1','1','0','1','0','0','1','1','1'] #prueba ['1','1','0','0','0','1','0']

        print("Señal Enviada:", dato_send)
        print("Señal Recibida:", dato_received,"\n")

        #deteccion de los errores
        resultado = detect_errores(dato_received, dato_send)

        errores = resultado[2]
        dato_corregido = resultado[1]
        error_vector = resultado [0] 

        if errores != 0:
            print("Se detectó un error. El dato corregido es:", dato_corregido)
        else:
            print("No se detectaron errores. Dato recibido:", dato_corregido)

        print("El Vector de Error es:", error_vector,"\n")
        error_vector_str = ''.join(str(x) for x in error_vector)
        with open("VectorError.txt","a") as f:
            np.savetxt(f, [error_vector], delimiter=" ", fmt="%s") #Guarda los datos procesados en formato .txt     

        #calculo de BER
        ber = BER(error_vector,errores)
        print(errores, "se detectaron errores.\n", "BER:", ber,"\n")
        with open("BER.txt","a") as f:             
                np.savetxt(f, [ber], fmt='%1.9f', delimiter=' ') #Guarda los datos procesados en formato .txt

        #decodificacion de los datos
        dato_BPM = decodificador(dato_corregido)
        print("Valor decodificado de BPM: ", dato_BPM,"\n","----------------------------------------------------------------------","\n")
        with open("BPM.txt","a") as f:             #Guarda los datos procesados en formato .txt
            np.savetxt(f, [dato_BPM], fmt='%1.0f', delimiter=' ')

Señal Enviada: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1']
Señal Recibida: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1'] 

No se detectaron errores. Dato recibido: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1']
El Vector de Error es: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

0 se detectaron errores.
 BER: 0.0 

Valor decodificado de BPM:  86 
 ---------------------------------------------------------------------- 

Señal Enviada: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1']
Señal Recibida: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1'] 

No se detectaron errores. Dato recibido: ['0', '0', '0', '0', '1', '0', '1', '0', '1', '1', '0', '0', '0', '1', '1']
El Vector de Error es: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 

0 se detectaron errores.
 BER: 0.0 

Valor decodificado de BPM:  86 
 -----------------------------------------

In [40]:
import pandas as pd
import os

def BER_RANGE(SNRmin, SNRmax, archivo):
    BERlist = []
    SNRlist = []
    k = 0
    
    for SNRi in range(SNRmin, SNRmax + 1):  # Incluyendo SNRmax en el rango
        with open(archivo, "r") as f:
            data = f.readlines()
        
        Datos_Codificados = ConvDec(data)
        RuidoAñadido = EjecucionCanal(Datos_Codificados, SNRi)  # Pasa los datos codificados por el canal AWGN
        
        with open("Canalconruido.txt", "r") as f:
            Ruidoso = f.readlines()
        
        RuidoVar = int(data[0])
        if RuidoVar == 7:
            k = 7
        elif RuidoVar == 15:
            k = 15

        BERtotal = 0
        
        for x in range(1, len(Ruidoso)):
            dato_send = data[x].strip().split()
            dato_received = Ruidoso[x].strip().split()
            resultado = detect_errores(dato_received, dato_send)
            errores = resultado[2]
            error_vector = resultado[0]
            ber = BER(error_vector, errores)
            BERtotal += ber
        
        BERprom = BERtotal / (len(Ruidoso) - 1)  # Se resta 1 para excluir la línea de RuidoVar
        BERlist.append(BERprom)
        SNRlist.append(SNRi)
        print(f"SNR: {SNRi}, BER: {BERprom}")
    
    DatosFinalesSNR = {'SNR': SNRlist, 'BER': BERlist}
    direc = os.getcwd()
    
    if k == 7:
        filename = os.path.join(direc, 'SNR74.xlsx')
        sheet_name = "SNR74"
    elif k == 15:
        filename = os.path.join(direc, 'SNR1511.xlsx')
        sheet_name = "SNR1511"
    
    DataframeDatos = pd.DataFrame(DatosFinalesSNR)
    
    # Cambio realizado aquí:
    with pd.ExcelWriter(filename) as writer:
        DataframeDatos.to_excel(writer, sheet_name=sheet_name, index=None)
    print(f"Datos guardados en {filename}")


  
            


In [41]:
# Llamada a la función con el archivo especificado y el rango de SNR
archivo = "datacod15.txt"
BER_RANGE(-10, 25, archivo)

Capacidad del canal es: 0.138 Mb/s

El canal se encuentra saturado

SNR: -10, BER: 0.12698412698412692
Capacidad del canal es: 0.171 Mb/s

El canal se encuentra saturado

SNR: -9, BER: 0.14285714285714277
Capacidad del canal es: 0.212 Mb/s

El canal se encuentra saturado

SNR: -8, BER: 0.1317460317460316
Capacidad del canal es: 0.262 Mb/s

El canal se encuentra saturado

SNR: -7, BER: 0.1391534391534391
Capacidad del canal es: 0.323 Mb/s

El canal se encuentra saturado

SNR: -6, BER: 0.12222222222222219
Capacidad del canal es: 0.396 Mb/s

El canal se encuentra saturado

SNR: -5, BER: 0.11746031746031742
Capacidad del canal es: 0.483 Mb/s

El canal se encuentra saturado

SNR: -4, BER: 0.13015873015873014
Capacidad del canal es: 0.586 Mb/s

El canal se encuentra saturado

SNR: -3, BER: 0.10264550264550266
Capacidad del canal es: 0.706 Mb/s

El canal se encuentra saturado

SNR: -2, BER: 0.12592592592592589
Capacidad del canal es: 0.843 Mb/s

El canal se encuentra saturado

SNR: -1, BER: 0